# Ayala Morales Mauricio
### No. de cuenta: 315332122

---

# 6. Modelos del lenguaje

## Práctica 6: Evaluación de modelos de lenguaje

**Fecha de entrega: 03 de Noviembre de 2024**

### Importación de módulos y bibliotecas

In [152]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import ngrams
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/mauricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


- Crear un par de modelos del lenguaje usando un **corpus en español**
    - Corpus: El Quijote
        - URL: https://www.gutenberg.org/ebooks/2000

Funciones para la creación de los modelos, preprocesamiento de las oraciones, probabilidad de oraciones y cálculo de la perplejidad

In [ ]:
def preprocess(sent: list[str]) -> list[str]:
    """
    Preprocess the sentence by removing punctuation, converting to lower case, and adding
    special tokens of beginning and end of sentence.

    :param list[str] sent: List of words in a sentence.
    :return list[str]: List of preprocessed words from the sentence.
    """
    result = [word.lower() for word in sent]
    result.append("<EOS>")
    result.insert(0, "<BOS>")
    return result

def ngram_model(corpus: list[list[str]], n: int) -> defaultdict[str, defaultdict[str, int]]:
    """
    Create a n-gram model from the corpus.

    :param list[list[str]] corpus: List of sentences.
    :param int n: Degree of the n-gram.
    :return defaultdict[str, defaultdict[str, int]]: Dictionary with the frequency of each n-gram.
    """

    model = defaultdict(lambda: defaultdict(lambda: 0))
    if n == 1:
        for sentence in corpus:
            n_grams = ngrams(preprocess(sentence), n)
            for w in n_grams:
                model[w] += 1
        return model

    if n == 2:
        for sentence in corpus:
            n_grams = ngrams(preprocess(sentence), n)
            for w1, w2 in n_grams:
                model[w1][w2] += 1
        return model

    for sentence in corpus:
        n_grams = ngrams(preprocess(sentence), n)
        for t in n_grams:
            model[t[:-1]][t[-1]] += 1
    return model

def calculate_sent_prob(model: defaultdict, sentence: list[str], n: int) -> float:
    """
    Calculates the probability of a sentence given a model.

    :param defaultdict[str, defaultdict[str, int]] model: Model of n-grams.
    :param str sentence: Sentence.
    :param int n: Degree of the n-gram.
    :return float: Probability of the sentence.
    """
    n_grams = ngrams(preprocess(sentence), n)
    p = 0.0
    for gram in n_grams:
        if n == 2:
            key = gram[0]
            value = gram[1]
            try:
                p += np.log(model[key][value])
            except:
                p += 0.0
            return p
        
        key = gram[:-1]
        value = gram[-1]

        try:
            p += np.log(model[key][value])
        except:
            p += 0.0
    return p

def perplexity(model: defaultdict, corpus: list[list[str]], n: int) -> float:
    """
    Calculates the perplexity of a model on a corpus.

    :param defaultdict[str, defaultdict[str, int]] model: Model of n-grams.
    :param list[list[str]] corpus: List of sentences.
    :param int n: Degree of the n-gram.
    :return float: Perplexity of the model.
    """
    pp = []
    for sentence in corpus:
    #1. Normalizamos y agregamos símbolos especiales:

    #Log perplexity calculada para cada oracion:
        log_prob = calculate_sent_prob(model, sentence, n)
        perplexity = -(log_prob/len(sentence)-1)
        pp.append(perplexity)


    #promedio de las log perplexity:
    return sum(pp) / len(pp)

Leyendo el archivo con el texto que se usará como corpus y tokenizando.

In [154]:
with open('ElQuijote_corpus.txt', 'r') as f:
    text = f.read()
    f.close()

corpus = [word_tokenize(sentence) for sentence in text.split('\n') if sentence.strip() != '']

- Modelo de n-gramas con `n = [2, 3]`
    - Hold out con `test = 30%` y `train = 70%`

Creando el modelo de Bi-grama y separando en datos de entrenamiento y prueba.

In [155]:
bigram_model = ngram_model(corpus, 2)

bi_train_data, bi_test_data = train_test_split(corpus, test_size=0.3)

print("Bi-gram train data", len(bi_train_data))
print("Bi-gram tests data", len(bi_test_data))

Bi-gram train data 67352
Bi-gram tests data 28866


Creando el modelo de Tri-grama y separando en datos de entrenamiento y prueba.

In [156]:
trigram_model = ngram_model(corpus, 3)

tri_train_data, tri_test_data = train_test_split(corpus, test_size=0.3)

print("Tri-gram train data", len(tri_train_data))
print("Tri-gram tests data", len(tri_test_data))

Tri-gram train data 67352
Tri-gram tests data 28866


- Evaluar los modelos y reportar la perplejidad de cada modelo
  - Comparar los resultados entre los diferentes modelos del lenguaje (bigramas, trigramas)

Calculando la perplejidad de cada modelo con sus respectivos conjuntos de prueba

In [159]:
bigram_perplexity = perplexity(bigram_model, bi_test_data, 2)
trigram_perplexity = perplexity(trigram_model, tri_test_data, 3)

print("Bigram Perplexity:", bigram_perplexity)
print("Trigram Perplexity:", trigram_perplexity)

Bigram Perplexity: 0.6076382610599397
Trigram Perplexity: -0.9249292139711859


- ¿Cual fue el modelo mejor evaluado? ¿Porqué?

    El modelo con mejor evaluación es el de Tri-gramas, pues obtuvo una menor perplejidad, es decir, un menor nivel de incertidumbre en sus predicciones. Esto se debe a que el n-grama utilizado es de un grado mayor, pues el conocer más palabras anterior nos da más información sobre qué palabras son más probables de aparecer después.

# Referencias

- Mucho del código mostrado fue tomado del trabajo de la Dr. Ximena Guitierrez-Vasques
- https://lena-voita.github.io/nlp_course/language_modeling.html